In [ ]:
import requests
import csv
import glob
import os
import gzip
import multiprocessing
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from tqdm.notebook import tqdm_notebook
from tqdm.notebook import tqdm as tqdmn

def make_folder_if_required(path):
    if not os.path.exists(path): 
        os.makedirs(path) 


In [ ]:
basepathSummary = "240325_reference_genomes/gbff"
datasets = ["viral","bacteria"]
summaryURL = "https://ftp.ncbi.nlm.nih.gov/genomes/refseq/{dataset}/assembly_summary.txt"
fastaOut = "240325_reference_genomes/fasta"





In [ ]:
files2download = {d:set() for d in datasets}

## Manually add Mammalian Reference Genomes
files2download.update({
    "mammal/pig":  ["http://ftp.ncbi.nlm.nih.gov/genomes/refseq/vertebrate_mammalian/Sus_scrofa/latest_assembly_versions/GCF_000003025.6_Sscrofa11.1/GCF_000003025.6_Sscrofa11.1_genomic.gbff.gz"],
    "mammal/human":{"http://ftp.ncbi.nlm.nih.gov/genomes/refseq/vertebrate_mammalian/Homo_sapiens/latest_assembly_versions/GCF_000001405.40_GRCh38.p14/GCF_000001405.40_GRCh38.p14_genomic.gbff.gz"},
    "mammal/bat":{"http://ftp.ncbi.nlm.nih.gov/genomes/refseq/vertebrate_mammalian/Rhinolophus_ferrumequinum/latest_assembly_versions/GCF_004115265.2_mRhiFer1_v1.p/GCF_004115265.2_mRhiFer1_v1.p_genomic.gbff.gz"}
          })


for dataset in datasets:
    url = summaryURL.format(dataset=dataset)
    ncbiresponse = requests.get(url)
    make_folder_if_required(os.path.join(basepathSummary,dataset))
    with gzip.open (os.path.join(basepathSummary,dataset+"_assembly_summary.txt.gz"),'wt') as summarygz:
        summarygz.write(ncbiresponse.text)
    with gzip.open(os.path.join(basepathSummary,dataset+"_assembly_summary_filtered.txt.gz"),'wt') as summaryFgz:
        lines = ncbiresponse.text.split("\n")
        lines[1] = lines[1].replace('# ','') # Fix tsv header
        #filtered csv-writer
        reader = csv.DictReader(lines[1:],delimiter=str("\t"))
        writer = csv.DictWriter(summaryFgz, fieldnames=reader.fieldnames)
        writer.writeheader()
        for row in reader:
            if row.get("genome_rep")=="Full" and row.get("assembly_level")=="Complete Genome":
                downloadpath = row["ftp_path"]+"/"+row["ftp_path"].split("/")[-1]+"_genomic.gbff.gz"
                downloadpath=downloadpath.replace("https://","http://")
                files2download[dataset].add(downloadpath)
                writer.writerow(row)

for key, s in files2download.items(): 
    print(key,"-", len(s))
                

# Download gbff.gz files

In [ ]:
def downloadGBFFgz(url,typ):
    
    filepath = os.path.join(basepathSummary,typ,url.split("/")[-1])
    try:
        r = requests.get(url, stream=True)
        make_folder_if_required(os.path.join(basepathSummary,typ))
        with open(filepath, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
        if r.status_code!=200:
            print (r.status_code, url, typ)
            return (r.status_code, url, typ)
        return None
    except Exception as e:
        print (e)
        print(url,typ)
        print("____")
        return(e,url,typ)
        

def star_downloadGBFFgz(args):
    return downloadGBFFgz(*args)
                

fileTup = [(url,typ) for typ, allurls in files2download.items() for url in allurls ]
with multiprocessing.Pool(processes=5) as pool:
    failed ={x for x in tqdmn(pool.imap(star_downloadGBFFgz,fileTup),total=len(fileTup)) if x }
    for x in failed:
        print("Failed:",x)


In [ ]:
print(len(failed))
for x in failed:
    print("wget", x[1],"-P" ,x[2])

# Make indexed Genomic Regions form gbff files

In [ ]:
class CODINGREGION:
    def __init__(self,refid,locustags,start,stop):
        self.refid = refid
        self.locustags = locustags
        self.start = start
        self.stop = stop
    def __ge__(self, other):
        assert self.refid == other.refid
        return self.stop >= other.stop
    def __gt__(self, other):
        assert self.refid == other.refid
        return self.stop > other.stop
    def __iadd__(self,other):
        assert (other in self)
        assert self.refid == other.refid
        self.start = min(self.start,other.start)
        self.stop = max(self.stop,other.stop)
        self.locustags += other.locustags
        return self
        
    def __contains__(self,other):
        return self.start<=other.start<=self.stop or self.start<=other.stop<=self.stop
    
    def __repr__(self):
        return f"CODINGREGION({self.refid},{self.locustags},{self.start},{self.stop})"
    
class GBFF_ENTRY:
    def __init__(self,entry):
        self.entry = entry
        self.seqid =f'{entry.annotations.get("accessions")[0]}.{entry.annotations.get("sequence_version")}'
        self.totallength = len(entry.seq)
        self.codingObjList = []
        self.taxid  = -1
        self.singeleFeatures = []
        for feature in entry.features:
            if feature.type == "source":
                taxid = [x[6:] for x in entry.features[0].qualifiers.get("db_xref",[]) if x.startswith("taxon:")][0]
                if taxid:
                    self.taxid = taxid
                
            if feature.type == "CDS":
                assert self.taxid != -1
                for seqElem in feature.location.parts:
                    locustags = feature.qualifiers['locus_tag'] if 'locus_tag' in feature.qualifiers else []
                    self.singeleFeatures.append(CODINGREGION(self.seqid,locustags,seqElem.nofuzzy_start,seqElem.nofuzzy_end))
        #Merge Features if overlapping
        self.singeleFeatures.sort()
        
        for x in self.singeleFeatures:
            if len(self.codingObjList) and (x in self.codingObjList[-1]):
                self.codingObjList[-1]+=x
            else:
                self.codingObjList.append(x)

    
    def get_coding(self,minsize = 300):
        return [(
            self.seqid,
            x.start,
            x.stop,
            "__".join(x.locustags),
            str(self.entry.seq[x.start:x.stop]),
            self.taxid
        ) for x in self.codingObjList  if (x.stop-x.start)>= minsize]
        
    
    def get_noncoding(self,minsize = 300):
        noncoding=[]
        # Add starting piece as non-coding, if starting piece >= minsize
        if self.codingObjList and minsize<=self.codingObjList[0].start:
           
            noncoding.append(
                (self.seqid,
                 0,
                 self.codingObjList[0].start,
                 f"{self.seqid}_noncoding_0",
                 str(self.entry.seq[0:self.codingObjList[0].start]),
                 self.taxid
                )
            )
        
        # Add middle pieces between cds as noncoding
        if len(self.codingObjList)>1:
            noncoding+= [
                (self.seqid,
                 self.codingObjList[i-1].stop,
                 self.codingObjList[i].start,
                 f"{self.seqid}_noncoding_{i}",
                 str(self.entry.seq[self.codingObjList[i-1].stop:self.codingObjList[i].start]),
                 self.taxid
                )
                
                for i in range(1,len(self.codingObjList)) if (self.codingObjList[i].start-self.codingObjList[i-1].stop)>=minsize]
        
        
        # Add Endpiece as non-coding, if Endpiece >= minsize
        if self.codingObjList and (self.totallength-self.codingObjList[-1].stop) >=minsize :
            noncoding.append(
                (self.seqid,
                 self.codingObjList[-1].stop,
                 self.totallength+1,
                 f"{self.seqid}_noncoding_{len(self.codingObjList)+2}"
                 ,str(self.entry.seq[self.codingObjList[-1].stop:self.totallength+1]),
                 self.taxid
                )
            )
        return noncoding


In [ ]:
from Bio.bgzf import BgzfWriter, BgzfReader
from glob import glob
from tqdm.notebook import tqdm as t
import gzip
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import multiprocessing
import os
import numpy as np
import random

In [ ]:
nprocesses=60
out= "./240325_reference_genomes/regions/"




def gbff2Regionslists(origin_file_tuple):
    origin, file = origin_file_tuple
    response = {"origin":origin,"cdslen":0,"noncdslen":0,"cdsreg":[],"noncdsreg":[]}
    samplecdslength=0
    samplenoncdslength=0
    global out
    outpath = os.path.join(out,f"{origin}_"+".".join(os.path.basename(file).split(".")[:-2]))+"region.bgzf"
    
    try:
        with gzip.open(file,"rt") as gbff:
            with BgzfWriter(outpath,"w") as data_out:
                response = []
                for entry in SeqIO.parse(gbff, "genbank"):
                    gbffRegions = GBFF_ENTRY(entry)
         
                    for region in gbffRegions.get_coding():
                        length = region[2]-region[1]
                        pretell = data_out.tell()
                        data_out.write("%s\t%d\t%d\t%s\t%s\t%s\n"%region)
                        response.append(f"{origin}\tcds\t{outpath}\t{length}\t{pretell}\n")
                        
                    for region in gbffRegions.get_noncoding():
                        length = region[2]-region[1]
                        pretell = data_out.tell()
                        data_out.write("%s\t%d\t%d\t%s\t%s\t%s\n"%region)
                        response.append(f"{origin}\tncds\t{outpath}\t{length}\t{pretell}\n")

        return("".join(response))
    except gzip.BadGzipFile as e:
        print(e)
        print(file)
    except EOFError as e:
        print(e)
        print(file)
    except Exception as e:
        print ("Other Error:",e,"in File",file)
        
        
        
        

def region_indexer(dataOutputFile,gbff_tupleList):
    total_lengths= dict()
    index = dict()
    with gzip.open(dataOutputFile,"wt") as index_out:
        with multiprocessing.Pool(processes=nprocesses) as pool:
            for response in t(pool.imap_unordered(gbff2Regionslists,gbff_tupleList),total=len(gbff_tupleList)):
                if response:
                    index_out.write(response)


    print("Done")




In [ ]:
gbff_files = [("mammal_bat",'240325_reference_genomes/gbff/mammal/bat/GCF_004115265.2_mRhiFer1_v1.p_genomic.gbff.gz'),
 ("mammal_pig",'240325_reference_genomes/gbff/mammal/pig/GCF_000003025.6_Sscrofa11.1_genomic.gbff.gz'),
 ("mammal_human",'240325_reference_genomes/gbff/mammal/human/GCF_000001405.40_GRCh38.p14_genomic.gbff.gz')]

gbff_files += [("viral",x) for x in glob("240325_reference_genomes/gbff/viral/*.gbff.gz") ]
gbff_files += [("bacteria",x) for x in glob("240325_reference_genomes/gbff/bacteria/*.gbff.gz") ]


regionINDEX = region_indexer("240325_reference_genomes/240328_regionindex.gz", gbff_files)


# Reload Index to Memory

In [ ]:
def load_index(indexfile):
    memory_index=dict()
    with gzip.open(indexfile,"rt") as idxf:
        for line in idxf:
            origin, cds_status, path,length,datastart= line.strip().split("\t")
            if origin not in memory_index:
                memory_index[origin] = {"cds":[],"ncds":[],"totalcds":0,"totalnoncds":0}
            memory_index[origin][cds_status].append( (int(length),path,int(datastart)))
            if cds_status=="cds":
                memory_index[origin]["totalcds"]+=int(length)
            else:
                memory_index[origin]["totalnoncds"]+=int(length)

    return memory_index
region_index = load_index("240325_reference_genomes/240328_regionindex.gz")


# Get Probability Vectors

In [ ]:
probvectors = {}
for origin, subdata in region_index.items():
    probvectors[origin]={}
    for cds_status in ["cds","ncds"]:
        totallength = subdata["totalcds"] if cds_status =="cds" else subdata["totalnoncds"] 
        print (totallength)
        probvectors[origin][cds_status] = np.asarray([data[0] for data in subdata[cds_status]],dtype=np.double)/totallength
print(probvectors)

# Define Sample Sizes

Training Data: Mammian per cds/noncds --> 166666 Reads
Test Data: Mammian per cds/noncds --> 1666 Reads

Viral/ Bacterial:
Training Data: 500000 Reads per cds/noncds
Test Data: 5000 Reads per cds/noncds

Draw Random from Kategories without replacement

In [ ]:
mammalsize = (166666,1666)
othersize = (500000,5000)
readlength = 300


# Small set
mammalsize = (83334,1666)
othersize = (250000,5000)
readlength = 300


In [ ]:
# Draw Order in final file

order_traindata = [zz for yy in [[(cls,cds_type)]*(mammalsize[0] if cls.startswith("mammal") else othersize[0]) for cls in region_index.keys() for cds_type in ["cds","ncds"]] for zz in yy]
order_testdata = [zz for yy in [[(cls,cds_type)]*(mammalsize[1] if cls.startswith("mammal") else othersize[1]) for cls in region_index.keys() for cds_type in ["cds","ncds"]] for zz in yy]
random.seed(42)
random.shuffle(order_traindata)
random.shuffle(order_testdata)
print(order_testdata[:10])

In [ ]:
#probvectors[origin][cds_status]
def randomnesgenerator ( probs, name = None, initlength=600000):
    while True:
        if name:
            print("Init:", name, "Generator")
        readIndex = np.random.choice(len(probs),initlength, p=probs)
        orientation = np.random.choice(2,initlength)
        for rI, orientation in zip(readIndex,orientation):
            yield rI,orientation
        if name:
            print("Draw new Batch:",name, "Generator")

    







class indexedRead:
    def __init__(self,read,orientation,readlength=300):
        length, path, datastart = read
        self.regionlength = length
        self.readlength = readlength
        self.orientation = orientation
        # Draw Start and Orientation
        
        self.readstartinregion = 0 if length == readlength else np.random.choice(self.regionlength-readlength,1)[0]
        with BgzfReader(path) as data:
            data.seek(datastart)
            d = data.readline().strip().split("\t")

            self.seqid, rstart,rstop,self.locus, regionSequence,self.taxid = d
            self.regionstart = int(rstart)
            self.regionstop  = int(rstop)
            self.readstartinref =  self.regionstart + self.readstartinregion
            

            self.readSeq = regionSequence[self.readstartinregion:(self.readstartinregion+readlength)].upper()

            self.has_no_ambiguities = not set(self.readSeq).difference({'A','T','C','G'})
            
            if orientation:
                self.readSeq = self.readSeq[::-1].translate({65: 'T', 84: 'A', 71: 'C', 67: 'G'})
            self.readblocks = " ".join([self.readSeq[index:index+6] for index in range(readlength-5)])

def readtuple2line(index,origin,cds_status,readlength = 300):
    # Draw single read
    timeout = 10000

    # Avoid Infinit Loop
    for i in range(timeout):
        singleReadFromIndexPOS, reversedRead =next(randgen[origin][cds_status])

        # Single Read Attributes
        read = index[origin][cds_status][singleReadFromIndexPOS]
        
        # Define DNA-BERT Label
        label = 1 if cds_status == "cds" else 0
        readentry = indexedRead(read,reversedRead)
        if readentry.has_no_ambiguities:
            return f"{readentry.readblocks}\t{label}\t{origin}\t{readentry.seqid}\t{readentry.taxid}\t{readentry.locus}\t{readentry.readstartinref}\t{readentry.regionstart}\t{readentry.regionstop}\t{reversedRead}\n"
            #return f"{readentry.readSeq}\t{label}\t{origin}\t{readentry.seqid}\t{readentry.taxid}\t{readentry.locus}\t{readentry.readstartinref}\t{readentry.regionstart}\t{readentry.regionstop}\t{reversedRead}\n"

    print(origin,cds_status,"timeout")
    assert False

def readsFromIndex2Tsv(index,readcats,outputfile):
    with gzip.open(outputfile,"wt") as output:
        output.write("Sequence\tLabel\tOrigin\tReferenceID\ttaxid\tLocus\tStartInReference\tRegionStartinRef\tRegionStopinRef\tReversed\n")
        for readclass in t(readcats):
            output.write(readtuple2line(index,*readclass))
               

In [ ]:
np.random.seed(42)
## init
randgen = {cls: {cds_type:randomnesgenerator(probs,f"{cls} {cds_type}") for cds_type, probs in v1.items()} for cls,v1 in probvectors.items() }
print(randgen)
    
readsFromIndex2Tsv(region_index,order_testdata,"240325_reference_genomes/traindata/dev_Smallk.tsv.gz")
readsFromIndex2Tsv(region_index,order_traindata,"240325_reference_genomes/traindata/train_Smallk.tsv.gz")

In [ ]:
np.random.seed(42)
## init
randgen = {cls: {cds_type:randomnesgenerator(probs,f"{cls} {cds_type}") for cds_type, probs in v1.items()} for cls,v1 in probvectors.items() }
print(randgen)
    
readsFromIndex2Tsv(region_index,order_testdata,"240325_reference_genomes/traindata/dev.tsv.gz")
readsFromIndex2Tsv(region_index,order_traindata,"240325_reference_genomes/traindata/train.tsv.gz")